## 📦 Setup & GitHub Repo Import

In [1]:
!git clone https://github.com/Techierookies/Real-time-competitor-strategy-tracker.git
!cd Real-time-competitor-strategy-tracker && ls -la

Cloning into 'Real-time-competitor-strategy-tracker'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 169 (delta 44), reused 161 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (169/169), 7.10 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (44/44), done.
total 40892
drwxr-xr-x 11 root root     4096 Oct 16 15:07 .
drwxr-xr-x  1 root root     4096 Oct 16 15:07 ..
-rw-r--r--  1 root root 14528512 Oct 16 15:07 competitor_tracker.db
drwxr-xr-x  2 root root     4096 Oct 16 15:07 database
drwxr-xr-x  2 root root     4096 Oct 16 15:07 data_collection
-rw-r--r--  1 root root      870 Oct 16 15:07 DATACOLLECTIONREADME.md
drwxr-xr-x  2 root root     4096 Oct 16 15:07 docs
-rw-r--r--  1 root root      621 Oct 16 15:07 eslint.config.js
drwxr-xr-x  2 root root     4096 Oct 16 15:07 exports
drwxr-xr-x  8 root root     4096 Oct 16 15:07 .git
-rw-r--r--  1 root root     5011 Oct 16 15:

In [2]:
!pip install tensorflow keras numpy pandas scikit-learn faker transformers torch
!pip install matplotlib seaborn plotly tabulate nltk textblob

import warnings
warnings.filterwarnings('ignore')
print("✅ All packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.1 MB/s eta 0:00:00
✅ All packages installed successfully!


In [3]:
import os
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import torch
from faker import Faker
import json
import random
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tabulate import tabulate
import string
import itertools
from collections import defaultdict
import shutil

np.random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)
random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")
print("✅ Libraries imported successfully!")

TensorFlow version: 2.19.0
PyTorch version: 2.8.0+cu126
✅ Libraries imported successfully!


## 🔍 Robust Database Analysis

In [4]:
db_path = 'Real-time-competitor-strategy-tracker/competitor_tracker.db'

if not os.path.exists(db_path):
    print(f"❌ Database file not found at: {db_path}")
    IPHONE_MODELS = ['iPhone 15', 'iPhone 16', 'iPhone 17']
    PRICE_RANGES = {
        'iPhone 15': (70835, 93640),
        'iPhone 16': (127494, 148357),
        'iPhone 17': (145000, 180000)
    }
    RATING_RANGES = {
        'iPhone 15': (4.3, 4.9),
        'iPhone 16': (4.1, 4.5),
        'iPhone 17': (4.5, 5.0)
    }
else:
    file_size = os.path.getsize(db_path)
    print(f"📁 Database file found: {db_path}")
    print(f"📊 File size: {file_size} bytes")

    try:
        conn = sqlite3.connect(db_path)
        tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
        tables_df = pd.read_sql_query(tables_query, conn)

        print(f"📋 Tables found: {len(tables_df)}")

        if len(tables_df) > 0:
            print(f"✅ Available tables: {tables_df['name'].tolist()}")

            iphone_models = set()
            real_price_ranges = {}
            real_rating_ranges = {}

            for table_name in tables_df['name']:
                try:
                    schema_query = f"PRAGMA table_info({table_name});"
                    schema = pd.read_sql_query(schema_query, conn)
                    print(f"\n📊 Table '{table_name}' columns: {schema['name'].tolist()}")

                    data_query = f"SELECT * FROM {table_name} LIMIT 10;"
                    data = pd.read_sql_query(data_query, conn)
                    print(f"   Rows in table: {len(data)}")

                    if 'model' in data.columns and 'price' in data.columns and 'rating' in data.columns:
                        for _, row in data.iterrows():
                            model_str = str(row['model'])
                            if 'iPhone' in model_str:
                                for model_num in ['15', '16', '17']:
                                    if model_num in model_str:
                                        model_name = f'iPhone {model_num}'
                                        iphone_models.add(model_name)

                                        try:
                                            price_val = float(str(row['price']).replace('₹', '').replace(',', ''))
                                            rating_val = float(row['rating'])

                                            if model_name not in real_price_ranges:
                                                real_price_ranges[model_name] = [price_val, price_val]
                                                real_rating_ranges[model_name] = [rating_val, rating_val]
                                            else:
                                                real_price_ranges[model_name][0] = min(real_price_ranges[model_name][0], price_val)
                                                real_price_ranges[model_name][1] = max(real_price_ranges[model_name][1], price_val)
                                                real_rating_ranges[model_name][0] = min(real_rating_ranges[model_name][0], rating_val)
                                                real_rating_ranges[model_name][1] = max(real_rating_ranges[model_name][1], rating_val)
                                        except:
                                            pass
                        break

                except Exception as e:
                    print(f"   ⚠️ Error reading table '{table_name}': {e}")

            conn.close()

            if iphone_models and real_price_ranges:
                IPHONE_MODELS = sorted(list(iphone_models))
                PRICE_RANGES = {model: tuple(ranges) for model, ranges in real_price_ranges.items()}
                RATING_RANGES = {model: tuple(ranges) for model, ranges in real_rating_ranges.items()}

                print(f"\n✅ Found iPhone models: {IPHONE_MODELS}")
                print(f"\n💰 Real Price Ranges:")
                for model, (min_p, max_p) in PRICE_RANGES.items():
                    print(f"   {model}: ₹{min_p:,.0f} - ₹{max_p:,.0f}")
                print(f"\n⭐ Real Rating Ranges:")
                for model, (min_r, max_r) in RATING_RANGES.items():
                    print(f"   {model}: {min_r:.1f} - {max_r:.1f}")
            else:
                IPHONE_MODELS = ['iPhone 15', 'iPhone 16', 'iPhone 17']
                PRICE_RANGES = {
                    'iPhone 15': (70835, 93640),
                    'iPhone 16': (127494, 148357),
                    'iPhone 17': (145000, 180000)
                }
                RATING_RANGES = {
                    'iPhone 15': (4.3, 4.9),
                    'iPhone 16': (4.1, 4.5),
                    'iPhone 17': (4.5, 5.0)
                }
        else:
            IPHONE_MODELS = ['iPhone 15', 'iPhone 16', 'iPhone 17']
            PRICE_RANGES = {
                'iPhone 15': (70835, 93640),
                'iPhone 16': (127494, 148357),
                'iPhone 17': (145000, 180000)
            }
            RATING_RANGES = {
                'iPhone 15': (4.3, 4.9),
                'iPhone 16': (4.1, 4.5),
                'iPhone 17': (4.5, 5.0)
            }

    except Exception as e:
        print(f"❌ Error accessing database: {e}")
        IPHONE_MODELS = ['iPhone 15', 'iPhone 16', 'iPhone 17']
        PRICE_RANGES = {
            'iPhone 15': (70835, 93640),
            'iPhone 16': (127494, 148357),
            'iPhone 17': (145000, 180000)
        }
        RATING_RANGES = {
            'iPhone 15': (4.3, 4.9),
            'iPhone 16': (4.1, 4.5),
            'iPhone 17': (4.5, 5.0)
        }

print(f"\n🎯 Final iPhone models: {IPHONE_MODELS}")
print("✅ Database analysis complete!")

📁 Database file found: Real-time-competitor-strategy-tracker/competitor_tracker.db
📊 File size: 14528512 bytes
📋 Tables found: 2
✅ Available tables: ['raw_scrapes', 'sqlite_sequence']

📊 Table 'raw_scrapes' columns: ['id', 'model', 'site', 'url', 'raw_html', 'scraped_at', 'price', 'rating', 'reviews']
   Rows in table: 10

✅ Found iPhone models: ['iPhone 15', 'iPhone 16', 'iPhone 17']

💰 Real Price Ranges:
   iPhone 15: ₹71,197 - ₹148,357
   iPhone 16: ₹74,869 - ₹129,933
   iPhone 17: ₹85,408 - ₹144,713

⭐ Real Rating Ranges:
   iPhone 15: 3.5 - 4.9
   iPhone 16: 4.1 - 4.4
   iPhone 17: 3.7 - 4.1

🎯 Final iPhone models: ['iPhone 15', 'iPhone 16', 'iPhone 17']
✅ Database analysis complete!


## 🎯 Enhanced Data Variation Components

In [5]:
fake = Faker(['en_US', 'en_IN', 'en_GB', 'en_AU'])
Faker.seed(None)

SITES = ['Amazon', 'Flipkart']
SITE_WEIGHTS = [0.6, 0.4]

def generate_realistic_price(model, base_ranges):
    base_min, base_max = base_ranges.get(model, (70000, 180000))

    market_factor = np.random.uniform(0.85, 1.15)
    seasonal_factor = np.random.uniform(0.90, 1.10)
    competition_factor = np.random.uniform(0.88, 1.12)

    adjusted_min = base_min * market_factor * seasonal_factor * competition_factor
    adjusted_max = base_max * market_factor * seasonal_factor * competition_factor

    base_price = np.random.uniform(adjusted_min, adjusted_max)
    price_noise = np.random.uniform(-500, 500)
    final_price = base_price + price_noise

    return max(base_min * 0.8, final_price)

def generate_realistic_rating(model, base_ranges):
    base_min, base_max = base_ranges.get(model, (4.0, 5.0))

    alpha, beta_param = 5, 2
    rating_factor = np.random.beta(alpha, beta_param)

    rating = base_min + (base_max - base_min) * rating_factor
    rating_noise = np.random.uniform(-0.05, 0.05)
    final_rating = rating + rating_noise

    return round(max(3.0, min(5.0, final_rating)), 1)

REVIEW_COMPONENTS = {
    'positive_adjectives': [
        'excellent', 'outstanding', 'amazing', 'fantastic', 'superb', 'brilliant',
        'incredible', 'remarkable', 'exceptional', 'wonderful', 'impressive',
        'great', 'good', 'nice', 'solid', 'decent', 'satisfying', 'reliable'
    ],
    'features': [
        'camera quality', 'battery life', 'performance', 'display', 'design',
        'build quality', 'user interface', 'speed', 'storage', 'connectivity',
        'sound quality', 'screen clarity', 'overall experience', 'functionality',
        'features', 'capabilities', 'responsiveness', 'durability'
    ],
    'contexts': [
        'daily use', 'gaming', 'photography', 'work', 'entertainment',
        'professional tasks', 'multimedia', 'social media', 'business',
        'personal use', 'family use', 'travel', 'outdoor activities'
    ],
    'sentiments': {
        'very_positive': [
            'Perfect for {context}.', 'Absolutely love the {feature}.',
            'Highly recommend for {context}.', '{adjective} {feature}.',
            'Best phone for {context}.', 'Incredible {feature} quality.'
        ],
        'positive': [
            'Great {feature} and good for {context}.',
            'Good phone with {adjective} {feature}.',
            'Satisfied with the {feature}.',
            'Nice {feature}, works well for {context}.'
        ],
        'mixed': [
            'Good {feature} but {negative_aspect} could be better.',
            '{adjective} phone overall, though {negative_aspect} is average.',
            'Decent for {context}, {feature} is good.'
        ]
    },
    'negative_aspects': [
        'battery life', 'price point', 'storage capacity', 'camera in low light',
        'charging speed', 'weight', 'size', 'software updates',
        'heat management', 'signal strength'
    ]
}

def generate_unique_review(model, rating):
    if rating >= 4.5:
        sentiment_type = 'very_positive'
    elif rating >= 4.0:
        sentiment_type = 'positive'
    else:
        sentiment_type = 'mixed'

    template = np.random.choice(REVIEW_COMPONENTS['sentiments'][sentiment_type])
    adjective = np.random.choice(REVIEW_COMPONENTS['positive_adjectives'])
    feature = np.random.choice(REVIEW_COMPONENTS['features'])
    context = np.random.choice(REVIEW_COMPONENTS['contexts'])
    negative_aspect = np.random.choice(REVIEW_COMPONENTS['negative_aspects'])

    review = template.format(
        adjective=adjective,
        feature=feature,
        context=context,
        negative_aspect=negative_aspect
    )

    return review

print("🎯 Enhanced variation components initialized!")
print(f"📱 Models: {IPHONE_MODELS}")
print(f"🏪 Sites: {SITES}")
print(f"📝 Review components: {len(REVIEW_COMPONENTS['positive_adjectives'])} adjectives")
print(f"🎭 Review templates: {sum(len(v) for v in REVIEW_COMPONENTS['sentiments'].values())} variations")

🎯 Enhanced variation components initialized!
📱 Models: ['iPhone 15', 'iPhone 16', 'iPhone 17']
🏪 Sites: ['Amazon', 'Flipkart']
📝 Review components: 18 adjectives
🎭 Review templates: 13 variations


## 🕒 Generate Dataset with Timestamps

In [6]:
def generate_unique_url(site, model, used_urls):
    max_attempts = 100
    attempt = 0

    while attempt < max_attempts:
        if site == 'Amazon':
            patterns = [
                f"https://www.amazon.in/dp/{fake.bothify('??########')}",
                f"https://www.amazon.in/Apple-{model.replace(' ', '-')}/dp/{fake.bothify('??########')}",
                f"https://www.amazon.in/gp/product/{fake.bothify('??########')}"
            ]
            url = np.random.choice(patterns)
        else:
            model_slug = model.lower().replace(' ', '-')
            patterns = [
                f"https://www.flipkart.com/apple-{model_slug}/p/{fake.bothify('itm????????')}",
                f"https://www.flipkart.com/{model_slug}/p/{fake.bothify('itm????????')}",
                f"https://www.flipkart.com/apple-{model_slug}-smartphone/p/{fake.bothify('itm????????')}"
            ]
            url = np.random.choice(patterns)

        if url not in used_urls:
            used_urls.add(url)
            return url

        attempt += 1

    timestamp = str(int(datetime.now().timestamp() * 1000000))[-8:]
    if site == 'Amazon':
        url = f"https://www.amazon.in/dp/{fake.bothify('??')}{timestamp}"
    else:
        url = f"https://www.flipkart.com/apple-{model.lower().replace(' ', '-')}/p/itm{timestamp}"

    used_urls.add(url)
    return url

def generate_diverse_records_with_datetime(target_records=2500):
    records = []
    used_urls = set()
    used_combinations = set()
    price_history = defaultdict(list)
    rating_history = defaultdict(list)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)

    print(f"📊 Generating {target_records} diverse records with timestamps...")
    print(f"📅 Time range: {start_date.date()} to {end_date.date()}")

    for i in range(target_records):
        max_attempts = 50
        attempt = 0

        while attempt < max_attempts:
            variation_seed = i * 13 + attempt * 7
            np.random.seed(variation_seed)

            if i < target_records // 3:
                model = IPHONE_MODELS[i % len(IPHONE_MODELS)]
            else:
                model_weights = [1.0] * len(IPHONE_MODELS)
                for idx, m in enumerate(IPHONE_MODELS):
                    model_count = sum(1 for r in records if r.get('Model') == m)
                    if model_count > target_records // len(IPHONE_MODELS):
                        model_weights[idx] *= 0.5

                model_weights = np.array(model_weights)
                model_weights /= model_weights.sum()
                model = np.random.choice(IPHONE_MODELS, p=model_weights)

            site_counts = {'Amazon': 0, 'Flipkart': 0}
            for r in records:
                site_counts[r.get('Source', 'Amazon')] += 1

            if site_counts['Amazon'] > site_counts['Flipkart'] + 50:
                site = 'Flipkart'
            elif site_counts['Flipkart'] > site_counts['Amazon'] + 50:
                site = 'Amazon'
            else:
                site = np.random.choice(SITES, p=SITE_WEIGHTS)

            # Generate realistic timestamp
            days_back = np.random.randint(0, 180)
            hours_back = np.random.randint(0, 24)
            minutes_back = np.random.randint(0, 60)
            scraped_at = end_date - timedelta(days=days_back, hours=hours_back, minutes=minutes_back)

            # Time-based price variations
            base_price = generate_realistic_price(model, PRICE_RANGES)

            if days_back > 120:
                time_factor = np.random.uniform(0.92, 1.08)
            elif days_back > 60:
                time_factor = np.random.uniform(0.96, 1.04)
            else:
                time_factor = np.random.uniform(0.98, 1.02)

            price = base_price * time_factor

            # Seasonal pricing effects
            month = scraped_at.month
            if month in [11, 12]:
                price *= np.random.uniform(0.85, 0.95)
            elif month in [1, 2]:
                price *= np.random.uniform(1.02, 1.08)
            elif month in [9, 10]:
                if model == 'iPhone 17':
                    price *= np.random.uniform(1.05, 1.15)
                else:
                    price *= np.random.uniform(0.88, 0.95)

            # Historical price awareness
            model_prices = price_history[model]
            if model_prices:
                min_diff = min(abs(price - existing) for existing in model_prices)
                if min_diff < 1000 and len(model_prices) > 10:
                    price_adjustment = np.random.choice([-1, 1]) * np.random.uniform(1000, 5000)
                    price += price_adjustment

            rating = generate_realistic_rating(model, RATING_RANGES)

            model_ratings = rating_history[model]
            if model_ratings and rating in model_ratings[-20:]:
                rating_adjustment = np.random.choice([-0.1, -0.2, 0.1, 0.2])
                rating = max(3.0, min(5.0, rating + rating_adjustment))
                rating = round(rating, 1)

            combo_key = (model, site, int(price/1000), rating)

            if combo_key not in used_combinations or attempt > 30:
                used_combinations.add(combo_key)
                break

            attempt += 1

        review = generate_unique_review(model, rating)
        url = generate_unique_url(site, model, used_urls)

        record = {
            'ID': i + 1,
            'Model': model,
            'Source': site,
            'Price': round(price, 2),
            'Rating': rating,
            'Reviews': review,
            'URL': url,
            'Scraped_At': scraped_at.strftime('%Y-%m-%d %H:%M:%S')
        }

        records.append(record)
        price_history[model].append(price)
        rating_history[model].append(rating)

        if (i + 1) % 500 == 0:
            print(f"   Generated {i + 1}/{target_records} records with timestamps...")

    np.random.seed(42)
    return records

# Generate timestamped dataset
print("🔄 Starting timestamped dataset generation...")
diverse_records = generate_diverse_records_with_datetime(target_records=2500)

df_diverse = pd.DataFrame(diverse_records)

print(f"\n✅ Timestamped dataset generated successfully!")
print(f"   Total records: {len(df_diverse)}")
print(f"   Columns: {list(df_diverse.columns)}")
print(f"   Date range: {df_diverse['Scraped_At'].min()} to {df_diverse['Scraped_At'].max()}")
print(f"   Models: {df_diverse['Model'].unique()}")
print(f"   Sources: {df_diverse['Source'].unique()}")
print(f"   Unique prices: {df_diverse['Price'].nunique()} / {len(df_diverse)} ({df_diverse['Price'].nunique()/len(df_diverse)*100:.1f}%)")
print(f"   Unique ratings: {df_diverse['Rating'].nunique()} / {len(df_diverse)} ({df_diverse['Rating'].nunique()/len(df_diverse)*100:.1f}%)")
print(f"   Unique reviews: {df_diverse['Reviews'].nunique()} / {len(df_diverse)} ({df_diverse['Reviews'].nunique()/len(df_diverse)*100:.1f}%)")
print(f"   Unique URLs: {df_diverse['URL'].nunique()} / {len(df_diverse)} ({df_diverse['URL'].nunique()/len(df_diverse)*100:.1f}%)")
print(f"   Unique timestamps: {df_diverse['Scraped_At'].nunique()} / {len(df_diverse)} ({df_diverse['Scraped_At'].nunique()/len(df_diverse)*100:.1f}%)")

🔄 Starting timestamped dataset generation...
📊 Generating 2500 diverse records with timestamps...
📅 Time range: 2025-04-19 to 2025-10-16
   Generated 500/2500 records with timestamps...
   Generated 1000/2500 records with timestamps...
   Generated 1500/2500 records with timestamps...
   Generated 2000/2500 records with timestamps...
   Generated 2500/2500 records with timestamps...

✅ Timestamped dataset generated successfully!
   Total records: 2500
   Columns: ['ID', 'Model', 'Source', 'Price', 'Rating', 'Reviews', 'URL', 'Scraped_At']
   Date range: 2025-04-19 16:15:07 to 2025-10-16 08:55:07
   Models: ['iPhone 15' 'iPhone 16' 'iPhone 17']
   Sources: [np.str_('Amazon') np.str_('Flipkart')]
   Unique prices: 2500 / 2500 (100.0%)
   Unique ratings: 16 / 2500 (0.6%)
   Unique reviews: 1047 / 2500 (41.9%)
   Unique URLs: 2500 / 2500 (100.0%)
   Unique timestamps: 2488 / 2500 (99.5%)


## 💾 Dataset Analysis & Export

In [7]:
# Save timestamped CSV
df_diverse.to_csv('enhanced_synthetic_dataset_with_timestamps.csv', index=False)

print("💾 Timestamped dataset saved as 'enhanced_synthetic_dataset_with_timestamps.csv'")
print("\n=== TIMESTAMPED DATASET STRUCTURE ===")
print(df_diverse.info())

print(f"\nTotal Rows: {len(df_diverse)}")
print(f"Total Columns: {len(df_diverse.columns)}")

print("\n=== TIME RANGE ANALYSIS ===")
df_diverse['Scraped_At'] = pd.to_datetime(df_diverse['Scraped_At'])
earliest = df_diverse['Scraped_At'].min()
latest = df_diverse['Scraped_At'].max()
date_span = (latest - earliest).days

print(f"Earliest record: {earliest}")
print(f"Latest record: {latest}")
print(f"Date span: {date_span} days")

print("\n=== FIRST 10 TIMESTAMPED RECORDS ===")
print(tabulate(df_diverse.head(10), headers='keys', tablefmt='psql', showindex=False))

print("\n=== TIME-BASED MODEL DISTRIBUTION ===")
model_dist = df_diverse['Model'].value_counts()
for model, count in model_dist.items():
    percentage = (count / len(df_diverse)) * 100
    model_data = df_diverse[df_diverse['Model'] == model]
    avg_price = model_data['Price'].mean()
    price_std = model_data['Price'].std()
    avg_rating = model_data['Rating'].mean()
    rating_std = model_data['Rating'].std()
    earliest_model = model_data['Scraped_At'].min()
    latest_model = model_data['Scraped_At'].max()
    print(f"   {model}: {count} records ({percentage:.1f}%)")
    print(f"      Price: ₹{avg_price:.0f} ± {price_std:.0f} | Rating: {avg_rating:.2f} ± {rating_std:.2f}")
    print(f"      Time span: {earliest_model.date()} to {latest_model.date()}")

print("\n=== MONTHLY TRENDS ===")
monthly_stats = df_diverse.groupby(df_diverse['Scraped_At'].dt.to_period('M')).agg({
    'Price': ['mean', 'count'],
    'Rating': 'mean'
}).round(2)

monthly_stats.columns = ['Avg_Price', 'Record_Count', 'Avg_Rating']
print(monthly_stats)

print("\n🎉 TIMESTAMPED DATASET GENERATION COMPLETE!")
print("✅ 2500+ timestamped records with 6-month historical span")
print("✅ Seasonal pricing effects and time-based variations")
print("✅ Ready for time-series analysis and forecasting!")

💾 Timestamped dataset saved as 'enhanced_synthetic_dataset_with_timestamps.csv'

=== TIMESTAMPED DATASET STRUCTURE ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          2500 non-null   int64  
 1   Model       2500 non-null   object 
 2   Source      2500 non-null   object 
 3   Price       2500 non-null   float64
 4   Rating      2500 non-null   float64
 5   Reviews     2500 non-null   object 
 6   URL         2500 non-null   object 
 7   Scraped_At  2500 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 156.4+ KB
None

Total Rows: 2500
Total Columns: 8

=== TIME RANGE ANALYSIS ===
Earliest record: 2025-04-19 16:15:07
Latest record: 2025-10-16 08:55:07
Date span: 179 days

=== FIRST 10 TIMESTAMPED RECORDS ===
+------+-----------+----------+----------+----------+-------------------------------------------------

## 🗄️ SQLite Export with Timestamps

In [11]:
# Complete working cell: Convert timestamps and export to SQLite with move to repo

import sqlite3
import shutil
import os
from datetime import datetime

# Ensure timestamps are plain strings
df_diverse['Scraped_At'] = df_diverse['Scraped_At'].astype(str)

# SQLite export
db_filename = 'synthetic_competitor_tracker.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS synthetic_competitor_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    model TEXT NOT NULL,
    site TEXT NOT NULL,
    url TEXT,
    raw_html TEXT,
    scraped_at TEXT NOT NULL,
    price REAL NOT NULL,
    rating REAL NOT NULL,
    reviews TEXT
)
""")

print(f"Inserting {len(df_diverse)} records with timestamps...")

for idx, row in df_diverse.iterrows():
    cursor.execute("""
        INSERT INTO synthetic_competitor_data
        (model, site, url, raw_html, scraped_at, price, rating, reviews)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['Model'],
        row['Source'],
        row['URL'],
        f'<synthetic>Generated on {datetime.now().strftime("%Y-%m-%d")}</synthetic>',
        row['Scraped_At'],
        row['Price'],
        row['Rating'],
        row['Reviews']
    ))
    if (idx + 1) % 500 == 0:
        print(f"   {idx + 1}/{len(df_diverse)} records inserted...")

conn.commit()

cursor.execute("SELECT COUNT(*) FROM synthetic_competitor_data")
record_count = cursor.fetchone()[0]
print(f"\nInserted {record_count} records with timestamps")

# Summary statistics
cursor.execute("""
    SELECT model, COUNT(*) as count, AVG(price) as avg_price, AVG(rating) as avg_rating,
           MIN(scraped_at) as earliest, MAX(scraped_at) as latest
    FROM synthetic_competitor_data
    GROUP BY model
""")
print("\nDatabase Statistics with Time Range:")
print("-" * 80)
for model, count, avg_price, avg_rating, earliest, latest in cursor.fetchall():
    print(f"{model:12s}: {count:4d} records | Price: ₹{avg_price:,.0f} | Rating: {avg_rating:.2f}")
    print(f"              Time range: {earliest} to {latest}")

# Move database and CSV to repo
conn.close()
repo_path = 'Real-time-competitor-strategy-tracker'
if os.path.exists(repo_path):
    shutil.copy2(db_filename, repo_path)
    shutil.copy2('enhanced_synthetic_dataset_with_timestamps.csv', repo_path)
    print(f"\nMoved DB and CSV to repository: {repo_path}")
else:
    print(f"\nRepository not found. Files saved in current directory.")

print("\n🗄️ SQLite export with timestamps complete!")

Inserting 2500 records with timestamps...
   500/2500 records inserted...
   1000/2500 records inserted...
   1500/2500 records inserted...
   2000/2500 records inserted...
   2500/2500 records inserted...

Inserted 2500 records with timestamps

Database Statistics with Time Range:
--------------------------------------------------------------------------------
iPhone 15   :  922 records | Price: ₹107,321 | Rating: 4.49
              Time range: 2025-04-20 01:43:07 to 2025-10-16 08:55:07
iPhone 16   :  762 records | Price: ₹101,747 | Rating: 4.31
              Time range: 2025-04-19 16:15:07 to 2025-10-16 07:42:07
iPhone 17   :  816 records | Price: ₹120,533 | Rating: 3.98
              Time range: 2025-04-19 19:37:07 to 2025-10-16 07:08:07

Moved DB and CSV to repository: Real-time-competitor-strategy-tracker

🗄️ SQLite export with timestamps complete!


In [13]:
from google.colab import files

# Replace with your file name (must exist in the current working directory)
filename = "enhanced_synthetic_dataset_with_timestamps.csv"

# Download the file
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>